In [56]:
import pandas as pd
import os
import numpy as np 
import joblib as joblib

In [89]:
# dataFiles = {dataFile :["./data/Parts_EQP_Output_ByMonth_20210407_van.csv"
#                 ,"./data/ScmTrainingData_Monthly_30days.csv"
#                 ,"./data/holiday.csv"
#                 ],
#             relation :[['MFG_MONTH','EQP_NO'],['MFG_MONTH','TOOL_ID']]
#                 ,[['MFG_MONTH'],['MFG_MONTH']]
#                ]

In [58]:
# datasetList =datafileList[0:1][0]

# datasetRels =datafileList[1:][0]
# datasetRels[0]

['MFG_MONTH', 'EQP_NO']

In [79]:
def readData(inputfile):
        df = pd.read_csv(inputfile)
        df=df.dropna(axis=1,how='all')
        df.info() 
        return analyzeData(df) 
def analyzeData(df):
        print('非數值欄位：')
        strColumnlist=df.select_dtypes(exclude=['int64','float64']).columns.tolist()
        print(strColumnlist) 
        print('數值欄位：')
        numbericColumnlist=df.select_dtypes(include=['int64','float64']).columns.tolist()
        print(numbericColumnlist)
        print('包含ＮＵＬＬ的欄位：')
        nullColumnlist=df.columns[df.isna().any()].tolist()    
        print(nullColumnlist)
        print('===================================================')  
        return df,strColumnlist,numbericColumnlist,nullColumnlist        
def mergeDataFrame(dfleft,dfright,LeftKeys,RightKeys):
        # dfright.columns = [str(col) + '_'+joinTableName for col in df.columns]
        df_merge = pd.merge(dfleft, dfright, left_on=LeftKeys, right_on=RightKeys,how="inner")
        return df_merge        

In [100]:
dataFiles = {'files':["./data/Parts_EQP_Output_ByMonth_20210407_van.csv"
                ,"./data/ScmTrainingData_Monthly_30days.csv"
                ,"./data/holiday.csv"
                ],
            'relations':[
                 [['MFG_MONTH','EQP_NO'],['MFG_MONTH','TOOL_ID']]
                ,[['MFG_MONTH'],['MFG_MONTH']]
               ]
}

In [103]:
dataFiles['files']

['./data/Parts_EQP_Output_ByMonth_20210407_van.csv',
 './data/ScmTrainingData_Monthly_30days.csv',
 './data/holiday.csv']

In [106]:


def merge(dataFiles):
    index =0
    for dfFile in dataFiles['files']:
        print(dfFile)
        if index ==0:
            _dfInputData1,_strColumnlist1,_numbericColumnlist1,_nullColumnlist1=readData(dfFile) 
            
            # _df_result.
        else:
            datasetRels = dataFiles['relations'][index-1]
            _dfInputData2,_strColumnlist2,_numbericColumnlist2,_nullColumnlist2=readData(dfFile) 
            _dfInputData1.set_index(datasetRels[0])
            _dfInputData2.set_index(datasetRels[1])
            df_merge = mergeDataFrame(_dfInputData1,_dfInputData2,datasetRels[0],datasetRels[1])
            _dfInputData1 = df_merge.copy(deep=False)
        index+=1
    return _dfInputData1 
df = merge(dataFiles)

./data/Parts_EQP_Output_ByMonth_20210407_van.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20775 entries, 0 to 20774
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PART_NO           20775 non-null  object 
 1   EQP_NO            20775 non-null  object 
 2   STOCK_EVENT_TIME  20775 non-null  object 
 3   QTY               20775 non-null  int64  
 4   PM                20775 non-null  float64
 5   TS                20775 non-null  float64
 6   ENG               20775 non-null  float64
 7   NST               20775 non-null  float64
 8   MFG_MONTH         20775 non-null  int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 1.4+ MB
非數值欄位：
['PART_NO', 'EQP_NO', 'STOCK_EVENT_TIME']
數值欄位：
['QTY', 'PM', 'TS', 'ENG', 'NST', 'MFG_MONTH']
包含ＮＵＬＬ的欄位：
[]
./data/ScmTrainingData_Monthly_30days.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1837 entries, 0 to 1836
Data columns (total 26 columns):

In [108]:
# _dfInputData1.reset_index(drop=True, inplace=True)
# _dfInputData1.drop(['index'],axis=1)
df.columns


Index(['PART_NO', 'EQP_NO', 'STOCK_EVENT_TIME', 'QTY', 'PM', 'TS', 'ENG',
       'NST', 'MFG_MONTH', 'TOOL_ID', 'M_NUM', 'UP_TIME', 'C_UP_TIME',
       'LOT_SIZE', 'C_LOT_SIZE', 'EQP_UTIL', 'C_EQP_UTIL', 'EQP_AVAIL_RATE',
       'TC', 'PROCESS_TIME', 'PROCESS_JOBTIME', 'INLINE_CT_LOT',
       'INLINE_CT_WAFER', 'MOVE_QTY', 'C_TC', 'ENG_LOT_RATE', 'HOT_LOT_RATE',
       'BACKUP_BY_RATE', 'BACKUP_FOR_RATE', 'RWORK_LOT_RATE', 'QUE_LOT_RATE',
       'SAMPLING_RATE', 'NUM_RECIPE', 'CHANGE_RECIPE', 'Holiday', 'Days'],
      dtype='object')

In [65]:
df_parts=pd.read_csv("./data/Parts_EQP_Output_ByMonth_20210407_2.csv")
df_parts['MFG_MONTH'] = df_parts['STOCK_EVENT_TIME'].apply(lambda x: pd.to_datetime(str(x), format='%Y/%m/%d %H:%M')).dt.strftime('%Y%m')
df_parts.to_csv("./data/Parts_EQP_Output_ByMonth_20210407_van.csv",index=False)